# 🎵 SoundFlow Premium AI Music Generator

**Enterprise-Grade Music Production on Google Colab**

This notebook generates premium AI music using:
- **MusicGen Stereo Large** (3.3B parameters)
- **Neuro-Symbolic DSP**: Binaural beats, sidechain compression
- **Professional Mastering**: -14 LUFS, 320kbps MP3

---

## ⚙️ Requirements

✅ **GPU Runtime**: T4 (free) or A100 (Colab Pro)  
✅ **RAM**: 12GB+  
✅ **Disk**: 20GB free  

**To enable GPU:**  
`Runtime > Change runtime type > Hardware accelerator > GPU`

---

In [ ]:
# ============================================================================
# STEP 1: Check GPU Availability
# ============================================================================

import torch

if not torch.cuda.is_available():
    raise RuntimeError(
        "❌ GPU not detected!\n"
        "Please enable GPU: Runtime > Change runtime type > GPU"
    )

gpu_name = torch.cuda.get_device_name(0)
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9

print("✅ GPU Available!")
print(f"🎮 GPU: {gpu_name}")
print(f"💾 VRAM: {gpu_memory:.1f} GB")

if gpu_memory < 15:
    print("\n⚠️  Warning: Less than 15GB VRAM. Consider using musicgen-stereo-medium instead.")

In [ ]:
# ============================================================================
# STEP 2: Install System Dependencies
# ============================================================================

!apt-get -qq update
!apt-get -qq install -y ffmpeg

print("✅ FFmpeg installed")

In [ ]:
# ============================================================================
# STEP 3: Clone Repository & Install Python Dependencies
# ============================================================================

# Clone your fork (replace with your GitHub URL)
!git clone https://github.com/ruslanmv/soundflow.git
%cd soundflow/generator

# Install dependencies
!pip install -q -r requirements-gpu.txt

print("✅ Dependencies installed")

In [ ]:
# ============================================================================
# STEP 4: Configure Generation (Edit This!)
# ============================================================================

from datetime import date

# Generation date (YYYY-MM-DD)
TODAY = date.today().strftime("%Y-%m-%d")

# Model Selection
# - "facebook/musicgen-stereo-large"  (3.3B params, best quality, 16GB VRAM)
# - "facebook/musicgen-stereo-medium" (1.5B params, good quality, 8GB VRAM)
MODEL = "facebook/musicgen-stereo-large"

# Duration override (optional, in seconds)
# Leave as None to use values from premium_prompts.yaml
DURATION = None  # e.g., 120 for 2 minutes

print(f"📅 Date: {TODAY}")
print(f"🤖 Model: {MODEL}")
print(f"⏱️  Duration: {DURATION or 'From config'}s")

In [ ]:
# ============================================================================
# STEP 5: Generate Premium Tracks
# ============================================================================

import sys

# Build command
cmd = f"python -m premium.musicgen_daily --date {TODAY} --model {MODEL}"
if DURATION:
    cmd += f" --duration {DURATION}"

print("🚀 Starting generation...\n")
print(f"Command: {cmd}\n")
print("="*70)

# Run generation
!{cmd}

print("\n" + "="*70)
print("✅ Generation complete!")

In [ ]:
# ============================================================================
# STEP 6: List Generated Tracks
# ============================================================================

from pathlib import Path
from IPython.display import Audio, display

output_dir = Path(".soundflow_out/premium")
tracks = sorted(output_dir.glob("*.mp3"))

print(f"📁 Found {len(tracks)} premium tracks:\n")

for i, track in enumerate(tracks, 1):
    size_mb = track.stat().st_size / (1024 * 1024)
    print(f"{i}. {track.name} ({size_mb:.1f} MB)")

# Play the first track
if tracks:
    print(f"\n🎧 Playing: {tracks[0].name}")
    display(Audio(tracks[0]))

In [ ]:
# ============================================================================
# STEP 7: Download All Tracks as ZIP
# ============================================================================

!zip -j premium_tracks.zip .soundflow_out/premium/*.mp3

from google.colab import files
files.download('premium_tracks.zip')

print("✅ Download started!")

---

## 🎨 Advanced: Custom Track Generation

Use this cell to generate a custom track with specific DSP settings:

In [ ]:
# ============================================================================
# Advanced: Custom Single Track
# ============================================================================

from premium.musicgen_daily import load_musicgen_model, configure_generation_params, generate_track_extended
from premium.postprocess import postprocess_track
from pathlib import Path

# Custom configuration
CUSTOM_PROMPT = "uplifting progressive house, 126 BPM, emotional melodic leads, euphoric"
CUSTOM_DURATION = 120  # 2 minutes

# DSP Flags
DSP_FLAGS = {
    "sidechain": True,
    "bpm": 126,
    "ducking_strength": 0.65,
    "fade_in_ms": 1000,
    "fade_out_ms": 2000,
}

# Load model (this may take a few minutes)
model = load_musicgen_model(MODEL, "cuda")
configure_generation_params(model, duration=30)

# Generate
sr, audio = generate_track_extended(model, CUSTOM_PROMPT, CUSTOM_DURATION, 30)

# Save & process
from audiocraft.data.audio import audio_write
import torch

output_dir = Path(".soundflow_out/premium")
tmp_dir = Path(".soundflow_tmp/premium")

tmp_path = tmp_dir / "custom_track"
final_path = output_dir / "custom_track.mp3"

# Save WAV
audio_tensor = torch.from_numpy(audio.T).unsqueeze(0)
audio_write(tmp_path, audio_tensor[0], sr, strategy="loudness", loudness_headroom_db=14)

# Post-process
postprocess_track(
    inp_wav=tmp_path.with_suffix(".wav"),
    out_mp3=final_path,
    target_lufs=-14.0,
    bitrate="320k",
    dsp_flags=DSP_FLAGS,
)

print(f"✅ Custom track saved: {final_path}")

# Play it
from IPython.display import Audio
display(Audio(final_path))